In [1]:
import os 
os.getcwd()

import numpy as np
import matplotlib.pyplot as plt

path_dataset='./data/'
standardize = False
min_max_scale = True
master = np.genfromtxt(path_dataset+'train.csv', delimiter=",", skip_header=1,
                       converters={1: lambda x: float(0) if b"b" in x else float(1)})
train_y, train_x = master[:, 1], np.delete(master, [0, 1], axis=1)

test_x = np.genfromtxt(path_dataset+'test.csv', delimiter=",", skip_header=1)
test_x = np.delete(test_x, [0, 1], axis=1)

if standardize == True:
    xmean, xstd = np.mean(train_x, axis=0), np.std(train_x, axis=0)
    train_x = (train_x - xmean) / xstd
    test_x = (test_x - xmean) / xstd
    
if min_max_scale == True:
    xmin, xmax = np.min(train_x, axis=0), np.max(train_x, axis=0)
    train_x = (train_x - xmin) / (xmax-xmin)
    test_x = (test_x - xmin) / (xmax-xmin)
    
y, x = train_y, train_x

def build_poly(x, degree):

    """
    Builds polynomial augmented dataset
    :param x: 
    :param degree: 
    :return:
    """
    r = x.copy()
    for deg in range (2,degree+1):
        r = np.c_[r, np.power(x, deg)]
        
    return np.c_[np.ones(r.shape[0]), r]

def build_k_indices(y, k_fold, seed):
    num_row = y.shape[0]
    interval = int(num_row/k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k*interval: (k+1) * interval] for k in range (k_fold)]
    return np.array(k_indices)

def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):

    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """

    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]

In [94]:
def compute_sigmoid(xw):

    """
    Computes sigmoid of input vector
    :param xw: (n,) array input to be sigmoid transformed
    :return: (n,) sigmoid-transformed vector
    """

    return 1 / (1 + np.exp(-xw))

def nlog_likelihood(y, tx, w):

    """
    Compute the negative log likelihood loss
    :param y: (n,) array
    :param tx: (n,d) matrix
    :param w: (d,) array of weights
    :return: computed loss given by negative log likelihood
    """
    pred = compute_sigmoid(tx.dot(w))
    pred = np.where(pred == 0, 0.000001, pred)
    pred = np.where(pred == 1, 0.999999, pred)
    loss = y.T.dot(np.log(pred)) + (1-y).T.dot(np.log(1-pred))
    return -loss

def compute_gradient_sigmoid(y, tx, w):

    """
    Compute sigmoid gradient
    :param y: (n,) array
    :param tx: (n,d) matrix
    :param w: (d,) array of initial weights
    :return: (d,) array of computed gradient vector
    """

    pred = compute_sigmoid(tx.dot(w))
    grd = tx.T.dot(pred-y)

    return grd

def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gammas, break_threshold):

    """
    Regularized logistic regression using SGD
    :param y: (n,) array
    :param tx: (n,d) matrix
    :param intial_w: (d,) array of initial weights
    :param max_iters: int indicating maximum iterations
    :param gamma: float indicating learning rate
    :return: optimal weights vector, loss(mse)
    """

    w = initial_w
    # uniform picking of minibatch of a single datapoint in this case
    for n_iter in range(max_iters):
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size=1, num_batches=1):
            break_count = 0
            
            # progressively reduce gamma size 
            if n_iter < 100: 
                gamma = gammas[0]
            elif n_iter < 750:
                gamma = gammas[1]
            elif n_iter < 1000:
                gamma = gammas[2]
            else:
                gamma = gammas[3]
                
            # retrieve gradient and cost
            grd = compute_gradient_sigmoid(minibatch_y, minibatch_tx, w) + lambda_ * w
            # update step
            w -= grd * gamma
            
            if n_iter > 1: 
                pl = l
            l = (nlog_likelihood(y, tx, w) + lambda_ * w)[0]
            if n_iter % 100 == 0: print(f'Iter {n_iter}: {l}')
                
            # break if the break_threshold (improvements) is met 5 times - idea is that we are no longer really improving    
            if n_iter > 1:
                if (np.abs(pl-l) < break_threshold):
                    break_count += 1
            if break_count>=4:
                break
        else:
            continue
        break
             
    loss = nlog_likelihood(y, tx, w)

    return w, loss

In [95]:
# just some weights i found from the unregularized log regression
weights_found = np.array([ 0.26142893,  0.86596468, -0.37762987,  0.37808003,  0.38866601,
        0.27821863,  0.49783078,  0.27445423,  1.06403986,  0.17753756,
        0.2306503 ,  0.02745172,  0.36734549,  0.27717549,  1.15019926,
        0.09041755,  0.25066148,  0.03816285,  0.08047934,  0.31177982,
        0.29509693,  0.14247221,  0.06900157,  0.03310452,  0.11632592,
        0.12584425,  0.1279479 ,  0.27926044,  0.27683506,  0.27699952,
        0.27397457,  0.21205993, -0.01097727, -1.04011751,  0.21621489,
       -0.40293883,  0.26377053, -0.41471766, -0.42243259,  0.12961204,
       -0.02179739, -0.19881119,  0.33088431, -0.40620022, -0.13329768,
       -0.15517697,  0.06508222, -0.09240051, -0.17634414,  0.11317156,
       -0.10073361, -0.01377299, -0.15074884, -0.26837094,  0.59101823,
        0.61882999,  0.6222557 , -0.40088948, -0.40726925, -0.40675704,
       -0.1286949 ])

In [96]:
# try training with different lambdas and refine the lambdas depending on what magnitude turns out to be best
# same idea with degrees

max_iters = 3000
gammas = [0.001, 0.0001, .00001, 0.000005]
initial_w = weights_found
lambdas = np.array([50,100,200])
degrees = np.arange(1,10)
seed = 12
k_fold = 7
break_threshold = 1

def cross_validation(y, x, k_indices, k, lambda_, degree, gammas, break_threshold):
    te_indice = k_indices[k]
    tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    tr_indice = tr_indice.reshape(-1)
    y_te, y_tr = y[te_indice], y[tr_indice]
    x_te, x_tr = x[te_indice], x[tr_indice]
    
    tx_tr = build_poly(x_tr, degree)
    tx_te = build_poly(x_te, degree)
    initial_w = np.ones(tx_tr.shape[1])
    
    w, _ = reg_logistic_regression(y_tr, tx_tr, lambda_, initial_w, max_iters, gammas, break_threshold)
    
    loss_tr = nlog_likelihood(y_tr, tx_tr, w)
    loss_te = nlog_likelihood(y_te, tx_te, w)
    
    return loss_tr, loss_te, w

def cross_validation_demo():
   
    
    k_indices = build_k_indices(y, k_fold, seed)


    best_lambdas = []
    best_losses = []
    best_weights = []

    
    j = 0 #jth degree
    for deg in degrees:
        i = 0 #ith lambda
        losses_te = []
        ws = []
        for lambda_ in lambdas:
            print(f'Lambda {lambdas[i]}, degree {degrees[j]}')
            i += 1
            ws_tmp = []
            loss_te_tmp = []

            for k in range(k_fold):
                print(f'Fold {k}/{k_fold}')
                loss_tr, loss_te, w = cross_validation(y, x, k_indices, k, lambda_, deg, gammas, break_threshold)
                loss_te_tmp.append(loss_te)
                ws_tmp.append(w)
            losses_te.append(np.mean(loss_te_tmp, axis=0))
            ws.append(np.mean(ws_tmp, axis=0))

        j += 1
        ind_lambda_opt = np.argmin(loss_te)
        best_lambdas.append(lambdas[ind_lambda_opt])
        best_losses.append(losses_te[ind_lambda_opt])
        best_weights.append(ws[ind_lambda_opt])
        
    ind_best_degree = np.argmin(best_losses)

    return degrees[ind_best_degree], best_lambdas, best_weights

best_degree, best_lambdas_per_degree, best_weights_per_degree = cross_validation_demo()

Lambda 50, degree 1
Fold 0/10
Iter 0: 1158971.6091759978
Iter 100: 156697.13534782888
Iter 200: 156254.43178298586
Iter 300: 155989.08283115827
Iter 400: 155843.90492568165
Iter 500: 155727.03244342128
Iter 600: 155731.089746107
Iter 700: 155728.69945278732
Iter 800: 155725.24667274658
Iter 900: 155725.3799464044
Iter 1000: 155726.21716607283
Iter 1100: 155721.49583055946
Iter 1200: 155720.732461245
Iter 1300: 155718.88117060502
Iter 1400: 155714.33889566414
Fold 1/10
Iter 0: 1159111.3475339634
Iter 100: 156720.10340406257
Iter 200: 156314.27519984602


KeyboardInterrupt: 